In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict

# Data sampling for the error analysis

## Statistics for error analysis

In [3]:
df = pd.read_csv('results_24shot_4126_PROMPT2_2_implicit_labels.csv')  
print(df)

                                                   post          class  \
0     bernie sanders : white people dont know what i...       not_hate   
1       " what do we do if ? " " - antifa attacks wi...       not_hate   
2                      the realist report - 9/11 debate       not_hate   
3     . #alert ; #splc has posted hitlists online th...       not_hate   
4     if other groups want to wine about optics and ...       not_hate   
...                                                 ...            ...   
4121  first isis is the jv team then they were conta...       not_hate   
4122  that is false. 2nd generation immigrants from ...  implicit_hate   
4123  racial identity  and its hostilities  are on t...       not_hate   
4124  as do the younger . what generation makes up a...       not_hate   
4125  pakistani sex-gang attacks covered up for fear...  implicit_hate   

         prediction implicit_class  
0     implicit_hate            NaN  
1          not_hate            NaN  


In [4]:
implicit_hate_df = df[df['class'] == 'implicit_hate'].copy()

print("=== IMPLICIT HATE PREDICTION STATISTICS BY IMPLICIT CLASS ===\n")

#unique implicit class labels
implicit_classes = implicit_hate_df['implicit_class'].unique()
implicit_classes = sorted([cls for cls in implicit_classes if pd.notna(cls)])

print(f"Total implicit hate instances: {len(implicit_hate_df)}")
print(f"Implicit class categories: {implicit_classes}\n")

#statistics for each implicit class
results = []

for implicit_class in implicit_classes:
    # Filter for current implicit class
    class_df = implicit_hate_df[implicit_hate_df['implicit_class'] == implicit_class]
    total_count = len(class_df)
    
    if total_count == 0:
        continue
    
    #predictions for each category
    pred_counts = class_df['prediction'].value_counts()
    implicit_pct = (pred_counts.get('implicit_hate', 0) / total_count) * 100
    explicit_pct = (pred_counts.get('explicit_hate', 0) / total_count) * 100
    not_hate_pct = (pred_counts.get('not_hate', 0) / total_count) * 100
    
    results.append({
        'implicit_class': implicit_class,
        'total_count': total_count,
        'implicit_hate_pct': implicit_pct,
        'explicit_hate_pct': explicit_pct,
        'not_hate_pct': not_hate_pct
    })
    
    print(f"{implicit_class.upper()} (n={total_count})")
    print(f"Predicted as implicit_hate: {implicit_pct:.1f}%")
    print(f"Predicted as explicit_hate: {explicit_pct:.1f}%")
    print(f"Predicted as not_hate: {not_hate_pct:.1f}%")
    print()

print("=== SUMMARY TABLE ===")
print(f"{'Implicit Class':<15} {'Total':<8} {'Implicit%':<10} {'Explicit%':<10} {'Not Hate%':<10}")
print("-" * 60)

for result in results:
    print(f"{result['implicit_class']:<15} {result['total_count']:<8} "
          f"{result['implicit_hate_pct']:<10.1f} {result['explicit_hate_pct']:<10.1f} "
          f"{result['not_hate_pct']:<10.1f}")

#general statistics
print(f"\n=== OVERALL PERFORMANCE ===")
total_implicit = len(implicit_hate_df)
overall_correct = len(implicit_hate_df[implicit_hate_df['prediction'] == 'implicit_hate'])
overall_as_explicit = len(implicit_hate_df[implicit_hate_df['prediction'] == 'explicit_hate'])
overall_as_not_hate = len(implicit_hate_df[implicit_hate_df['prediction'] == 'not_hate'])

print(f"Overall implicit hate detection rate: {(overall_correct/total_implicit)*100:.1f}%")
print(f"Overall misclassified as explicit: {(overall_as_explicit/total_implicit)*100:.1f}%")
print(f"Overall misclassified as not hate: {(overall_as_not_hate/total_implicit)*100:.1f}%")

#best and worst performing categories
if results:
    best_detection = max(results, key=lambda x: x['implicit_hate_pct'])
    worst_detection = min(results, key=lambda x: x['implicit_hate_pct'])
    
    print(f"\nBest detected implicit class: {best_detection['implicit_class']} ({best_detection['implicit_hate_pct']:.1f}%)")
    print(f"Worst detected implicit class: {worst_detection['implicit_class']} ({worst_detection['implicit_hate_pct']:.1f}%)")

### The code was created with the help of Claude.ai


=== IMPLICIT HATE PREDICTION STATISTICS BY IMPLICIT CLASS ===

Total implicit hate instances: 1250
Implicit class categories: ['incitement', 'inferiority', 'irony', 'other', 'stereotypical', 'threatening', 'white_grievance']

INCITEMENT (n=248)
Predicted as implicit_hate: 42.7%
Predicted as explicit_hate: 35.9%
Predicted as not_hate: 21.4%

INFERIORITY (n=169)
Predicted as implicit_hate: 38.5%
Predicted as explicit_hate: 24.9%
Predicted as not_hate: 36.7%

IRONY (n=156)
Predicted as implicit_hate: 42.9%
Predicted as explicit_hate: 21.2%
Predicted as not_hate: 35.9%

OTHER (n=16)
Predicted as implicit_hate: 62.5%
Predicted as explicit_hate: 18.8%
Predicted as not_hate: 18.8%

STEREOTYPICAL (n=221)
Predicted as implicit_hate: 48.4%
Predicted as explicit_hate: 35.3%
Predicted as not_hate: 16.3%

THREATENING (n=130)
Predicted as implicit_hate: 9.2%
Predicted as explicit_hate: 56.9%
Predicted as not_hate: 33.8%

WHITE_GRIEVANCE (n=301)
Predicted as implicit_hate: 56.5%
Predicted as explicit

## Random data sampling for error analysis (100 sentences per class)

In [29]:
def sample_csv_data(input_file, output_file, samples_per_class=100, max_samples=16):
    """
    Sample CSV data with equal distribution across all 3 prediction labels for each implicit_class.
    
    Args:
        input_file (str): Path to input CSV file
        output_file (str): Path to output CSV file
        samples_per_class (int): Number of samples per implicit_class (default: 100)
    """
    df = pd.read_csv(input_file)
    implicit_classes = df['implicit_class'].unique()
    all_predictions = ['explicit_hate', 'implicit_hate', 'not_hate']  # All 3 expected prediction labels
    
    sampled_data = []
    sampling_report = defaultdict(lambda: defaultdict(int))
    
    for implicit_class in implicit_classes:
        
        class_data = df[df['implicit_class'] == implicit_class].copy()
        
        if len(class_data) == 0:
            print(f"  Warning: No data available for implicit_class '{implicit_class}', skipping...")
            continue

        if implicit_class == 'other':
            target_samples = min(len(class_data), 16) # Max 16 for 'other' class
            if len(class_data) != max_samples:
                print(f"Warning: Expected {max_samples} samples but found {len(class_data)} samples")
        if len(class_data) < max_samples:
            print(f"Error: Not enough data! Need {max_samples} samples but only {len(class_data)} available")
            return None
        else:
            target_samples = samples_per_class  # 100 for other classes
        
        samples_per_prediction = target_samples // 3  # divide by 3 (all prediction labels)
        remainder = target_samples % 3

        if remainder > 0:
        
            class_samples = []
        
        for i, prediction in enumerate(all_predictions):
            n_samples = samples_per_prediction
            if i < remainder: 
                n_samples += 1
            
            pred_data = class_data[class_data['prediction'] == prediction]
            available_samples = len(pred_data)
                        
            if available_samples == 0:
                print(f"    Warning: No samples available for prediction '{prediction}'")
                continue
            
            if available_samples >= n_samples:
                sampled = pred_data.sample(n=n_samples, random_state=42)
            else:
                print(f"    Warning: Only {available_samples} samples available, taking all available samples")
                sampled = pred_data.copy()  # Take all available samples
            
            class_samples.append(sampled)
            sampling_report[implicit_class][prediction] = len(sampled)
        
        if class_samples:
            class_combined = pd.concat(class_samples, ignore_index=True)
            sampled_data.append(class_combined)
    
    if sampled_data:
        final_df = pd.concat(sampled_data, ignore_index=True)
        
        final_df = final_df.sample(frac=1, random_state=42).reset_index(drop=True)
        final_df.to_csv(output_file, index=False)
        
        return final_df
    
    else:
        print("No data was sampled. Please check your input file and criteria.")
        return None

if __name__ == "__main__":
    input_csv = "results_24shot_4126_PROMPT2_2_implicit_labels.csv"  # Replace with your input file path
    output_csv = "100_randomly_sampled_data.csv"  # Replace with desired output file path
    
    try:
        result = sample_csv_data(input_csv, output_csv, samples_per_class=100)
        
        if result is not None:
            print(f"\nSampling completed successfully!")
            print(f"Check the output file: {output_csv}")
        
    except FileNotFoundError:
        print(f"Error: Input file '{input_csv}' not found.")
        print("Please update the 'input_csv' variable with the correct file path.")
    except Exception as e:
        print(f"Error: {e}")

### The code was created with the help of Claude.ai


Sampling completed successfully!
Check the output file: 100_randomly_sampled_data.csv


In [30]:
df = pd.read_csv('100_randomly_sampled_data.csv')
df['implicit_class'].value_counts()

implicit_class
incitement         100
stereotypical      100
white_grievance    100
inferiority        100
irony               99
threatening         79
other               16
Name: count, dtype: int64

In [32]:
df = pd.read_csv('100_randomly_sampled_data.csv')
counts = df.groupby('implicit_class')['prediction'].value_counts().unstack(fill_value=0)
print(counts)

prediction       explicit_hate  implicit_hate  not_hate
implicit_class                                         
incitement                  34             33        33
inferiority                 34             33        33
irony                       33             33        33
other                        3             10         3
stereotypical               34             33        33
threatening                 34             12        33
white_grievance             34             33        33
